## Import

In [ ]:
from numpy import *
from matplotlib import pyplot as plt
from scipy.sparse import rand as sprand
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import eigs as speigs
from tqdm import trange

## Helper functions

In [ ]:
def normalize_input(inputSequence):
    iMu = mean(inputSequence, axis=0)
    iMax = amax(abs(inputSequence), axis=0)
    normInputSequence = (inputSequence-iMu)/iMax
    return normInputSequence

def normalized_mse(pred, true):
    # true (nSamples x nJoints)
    # pred (nSamples x nJoints)
    #return error_joints, error_avg
    
    # Normalizing with variance
    nSamples = size(true,axis=0);
    sigma = var(true,axis=0);
    
    # Calculating each error value
    es = abs(true-pred)**2
    s = sum(es,axis=0)
    error_joints = s/(nSamples*sigma) # Mean error of each joint
    error_avg = mean(error_joints,axis=0); # Combined joint error
    
    return error_joints, error_avg

#maxInput = array([]) # Fill with max (reasonable) ALSO ADD TO INIT IF USING!

## PC-ESN Class

In [ ]:
class PCESN():
    def __init__(self,nInputUnits,nReservoirUnits,nOutputUnits,spectralRadius,sigma2,phi2,smoothFactor):
        print('Generating PC-ESN...')
        # Structure
        self.nInputUnits = nInputUnits
        self.nReservoirUnits = nReservoirUnits
        self.nOutputUnits = nOutputUnits
        
        # Parameters
        self.spectralRadius = spectralRadius # 0<sR<1 to ensure ESP! default 0.4???
        self.sigma2 = sigma2
        self.phi2 = phi2
        self.eta = 1 # learning rate,  0.1 -> 0.01 eta^p < inf, sum(eta) = inf, 
        
        # Initalize input sum and mean variables
        self.inputMean = zeros((nInputUnits,1))
        #self.maxTorque = maxTorque
        self.t = 0

        # Initialize weights, H.Jaeger (Sparse reservoir weights)
        success = 0                                             
        while success == 0: # following block might fail
            try:
                self.Wres = sprand(nReservoirUnits, nReservoirUnits, density=10/nReservoirUnits)
                self.Wres = self.Wres.toarray()
                self.Wres[self.Wres!=0] -= 0.5 # modify only nonzero elements
                self.Wres = csr_matrix(self.Wres) # back to sparse
                maxVal = max(abs(speigs(A=self.Wres, k=1, which='LM')[0]))
                self.Wres /= maxVal
                success = 1
            except:
                success = 0   
        self.Wres *= self.spectralRadius
        
        self.Win = eye(nInputUnits)
        self.Wself = (2.0*random.rand(nReservoirUnits, nInputUnits)-1.0) # init not mentioned???
        self.Wfb = (2.0 * random.rand(nReservoirUnits, nOutputUnits)- 1.0)
        self.Wout = zeros((nOutputUnits,nReservoirUnits))
        self.Wdir = zeros((nOutputUnits,nInputUnits))
        self.Wtrain = hstack((self.Wout,self.Wdir)) # just a combination of [Wout Wdir]
        
        self.r = zeros((nReservoirUnits,1))
        self.s = zeros((nInputUnits,1))
        self.o = zeros((nOutputUnits,1))
        self.c = vstack((self.r, self.s))
        
        self.smoothFactor = smoothFactor # If ~1 trust new data, if ~0 rely on previous data
        
        #self.outputMean = zeros((nOutputUnits,1))
        #self.M2 = zeros((nOutputUnits,1))

        # Initialize covariance matrix
        self.V = self.phi2 * eye(nInputUnits + nReservoirUnits)
        
        print('Successful!\n')
        
    def train(self, inputSample,targetSample):
        """Training network on inputs with additional help of target values.

        Args:
            inputSample: numpy array (inputs x 1)
            targetSample: numpy array (inputs x 1)
        Returns:
            Updated parameters (weights and states)

        """
        
        # Center input
        self.inputMean = self.inputMean + (inputSample-self.inputMean)/(self.t+1)
        inputSample -= self.inputMean
        
        # Normalize with pre-determined max values (NOT PART OF ALGORITHM):
        #inputSample /= self.maxInput
        
        self.t += 1 # update time index

        self.s = tanh(self.Win @ inputSample) # update self-organized layer
        self.r = tanh(self.Wres @ self.r + self.Wself @ self.s + self.Wfb @ self.o)
        ccurr = self.c
        self.c = vstack((self.r, self.s))
        self.o = self.Wtrain @ self.c
        
        # Simple smoothing (NOT PART OF ALGORITHM)
        oldOut = self.o
        self.o = (1-self.smoothFactor)*oldOut + self.smoothFactor*(self.Wtrain @ self.c)

        Vprev = self.V
        self.V = linalg.inv(linalg.inv(Vprev) + (1/self.sigma2) * ccurr @ ccurr.T)
        a = self.V @ linalg.inv(Vprev) @ self.Wtrain.T
        b = 1/self.sigma2 * (self.V @ ccurr) @ targetSample.T
        
        #self.Wtrain = sum([a.T, b.T], axis=0)
        self.Wtrain = a.T + b.T

        # Calculate GHL update 
        dWin = self.eta*(inputSample@self.s.T - tril(self.s @ self.s.T) @ self.Win)
        self.Win += dWin # update Win matrix
        
        # Update learning rate (NOT PART OF ALGORITHM)
        self.eta = 1/sqrt(self.t)
        
        # Welford's Online variance algorithm (NOT PART OF ALGORITHM)
        #oldM = self.outputMean;
        #delta = targetSample-self.outputMean;
        #self.outputMean = self.outputMean + delta/obj.t;
        #self.M2 = self.M2 + (targetSample-self.outputMean) * delta;
        #self.sigma2 = vstack((ones(self.nReservoirUnits,1),matlib.repmat(self.M2/self.t,(3,1))))

# Demo

In [ ]:
nInputs = 21
nOutputs = 7
split = 0.5

data = genfromtxt('train_data2.csv', delimiter=',')
# 'baxter_rand'
# 'sarcos'
ind = round(split*data.shape[0])

uTrain = data[0:ind,0:nInputs,newaxis]
yTrain = data[0:ind,nInputs:,newaxis]
uTest = data[ind:,0:nInputs,newaxis]
yTest = data[ind:,nInputs:,newaxis]

# Normalize input
uTrain = normalize_input(uTrain)
uTest = normalize_input(uTest)

trainSamples = uTrain.shape[1]
testSamples = uTest.shape[1]

In [ ]:
nInputUnits = 21
nReservoirUnits = 600
nOutputUnits = 7
spectralRadius = 0.3 # Tune!!!
sigma2 = 0.1
phi2 = 1
smoothFactor = 0.9

pcesn = PCESN(nInputUnits,nReservoirUnits,nOutputUnits,spectralRadius,sigma2,phi2,smoothFactor)

totalSamples = 3000 # Polydoros model seems to converge in 2000 samples!
output = zeros((7,totalSamples,1))

for i in trange(totalSamples): # or standard "range" to remove progress bar 
    pcesn.train(uTrain[i,:],yTrain[i,:])
    output[:,i] = pcesn.o
    

In [ ]:
plt.figure(figsize=(12,15))
for j in range(1,8):
    plt.subplot(420+j)
    plt.plot(output[j-1,1500:2000],'b')
    plt.plot(yTrain[1500:2000,j-1],'r')
    plt.title('Joint {}'.format(j))
    

In [ ]:
error_joints, error_avg = normalized_mse(output[:,2700:totalSamples,0].T, yTrain[2700:totalSamples,:,0])
print(error_joints)
print(error_avg)

In [ ]:
output_test = zeros((7,testSamples,1))

for i in trange(testSamples): # or standard "range" to remove progress bar 
    pcesn.train(uTest[i,:],yTest[i,:])
    output_test[:,i] = pcesn.o
    